In [61]:
import gym
import numpy as np
import time 

In [62]:
env = gym.make('CartPole-v0')
print(env.action_space) #[Output: ] Discrete(2)
print(env.observation_space) # [Output: ] Box(4,)

Discrete(2)
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [63]:
# returns an initial observation
env.reset()

for i in range(10):

  # env.action_space.sample() produces either 0 (left) or 1 (right).
  action = env.action_space.sample()
  observation, reward, done, info = env.step(action)

  print("step", i, ":", action, observation, reward, done, info)
  if done:
    print("Finished after {} timesteps".format(i+1))
    break

env.close()

step 0 : 0 [-0.01246119 -0.22411801 -0.03466767  0.2518342 ] 1.0 False {}
step 1 : 1 [-0.01694356 -0.02851861 -0.02963099 -0.05157884] 1.0 False {}
step 2 : 1 [-0.01751393  0.16701542 -0.03066257 -0.35346135] 1.0 False {}
step 3 : 1 [-0.01417362  0.36255965 -0.03773179 -0.65565324] 1.0 False {}
step 4 : 0 [-0.00692243  0.16798276 -0.05084486 -0.37508613] 1.0 False {}
step 5 : 0 [-0.00356277 -0.02638151 -0.05834658 -0.09885845] 1.0 False {}
step 6 : 1 [-0.0040904   0.16952603 -0.06032375 -0.40936387] 1.0 False {}
step 7 : 0 [-0.00069988 -0.0246911  -0.06851102 -0.1362925 ] 1.0 False {}
step 8 : 0 [-0.0011937  -0.21876827 -0.07123688  0.13401382] 1.0 False {}
step 9 : 0 [-0.00556907 -0.41280136 -0.0685566   0.40339932] 1.0 False {}


In [64]:
def Qtable(state_space,action_space,bin_size = 30):
    
    bins = [np.linspace(-4.8,4.8,bin_size),
            np.linspace(-4,4,bin_size),
            np.linspace(-0.418,0.418,bin_size),
            np.linspace(-4,4,bin_size)]
    
    q_table = np.zeros(([bin_size] * state_space + [action_space]))
    #q_table = np.random.uniform(low=-1,high=1,size=([bin_size] * state_space + [action_space]))
    
    return q_table, bins

In [65]:
bin_size = 30
Q, bins = Qtable(4,2,bin_size)
Q.shape

(30, 30, 30, 30, 2)

In [66]:
def Discrete(state, bins):
    index = []
    for i in range(len(state)): index.append(np.digitize(state[i],bins[i]) - 1)
    return tuple(index)

In [67]:
test_state = env.reset()
test_state_discrete = Discrete(test_state, bins)
print(test_state)
print(test_state_discrete)

[-0.0452775  -0.03027727  0.03163901 -0.0296753 ]
(14, 14, 15, 14)


In [68]:
def Q_learning(q_table, bins, episodes = 500, gamma = 0.95, lr = 0.1, timestep = 100, epsilon = 0.2):
    
    runs = [0]
    data = {'max' : [0], 'avg' : [0]}
    rewards = 0
    steps = 0
    solved = False

    for episode in range(1,episodes+1):
        ep_start = time.time()
        steps += 1 
        # env.reset() => initial observation
        current_state = Discrete(env.reset(),bins)
      
        score = 0
        done = False
        while not done:
            if episode%timestep==0: env.render()
            if np.random.uniform(0,1) < epsilon:  # if random number is less than epsilon, choose random action
                    action = env.action_space.sample()
            else: # otherwise choose max reward action
                action = np.argmax(q_table[current_state])

            observation, reward, done, __ = env.step(action)  # take chosen action
            new_state = Discrete(observation,bins)
            score += reward  #???????
            
            if not done:
                max_future_q = np.max(q_table[new_state])  # best estimated reward at current state
                current_q = q_table[current_state+(action,)]
                new_q = (1-lr)*current_q + lr*(reward + gamma*max_future_q)
                q_table[current_state+(action,)] = new_q

            current_state = new_state
            
        # End of the loop update
        else:
            rewards += score
            runs.append(score)

            if score > 195 and steps >= 100 and solved == False:
                solved = True
                print('First solved in episode : {} in time {}'.format(episode, (time.time()-ep_start)))
        
        # Timestep value update
        if episode % timestep == 0:
            print('Episode : {} | Avg. Reward -> {} | Max reward : {} | Time : {}'.format(episode, rewards/timestep, max(runs), time.time() - ep_start))
            data['max'].append(max(runs))
            data['avg'].append(rewards/timestep)

            rewards, runs= 0, [0] 

In [69]:
Q_learning(Q, bins, episodes = 10000, gamma = 0.95, lr = 0.1, timestep = 1000, epsilon = 0.2)

First solved in episode : 478 in time 0.01296544075012207


Exception ignored in: <function Viewer.__del__ at 0x000001B802870280>
Traceback (most recent call last):
  File "C:\Users\sdien\Anaconda3\envs\NC\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\Users\sdien\Anaconda3\envs\NC\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\Users\sdien\Anaconda3\envs\NC\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\sdien\Anaconda3\envs\NC\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\sdien\Anaconda3\envs\NC\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x000001B87A6FFB30; to 'Win32Window' at 0x000001B87A8BBB80>


Episode : 1000 | Avg. Reward -> 60.868 | Max reward : 200.0 | Time : 4.495880842208862


KeyboardInterrupt: 